# 6) Variantes do BERT

Desde 2018, surgiram diversas versões otimizadas do BERT com objetivos diferentes —  
melhorar desempenho, reduzir custo ou mudar o pré-treinamento.

| Modelo | Principais características | Ganhos |
|---------|----------------------------|---------|
| **BERT-base / BERT-large** | Modelo original (12 / 24 camadas, 110M / 340M parâmetros). | Base de referência. |
| **DistilBERT** | 40% menor, 60% mais rápido, via *knowledge distillation*. | Eficiência. |
| **RoBERTa** | Remove NSP, usa *dynamic masking*, treina em muito mais dados. | +Robusto, melhor generalização. |
| **ALBERT** | Parâmetros compartilhados + fatorização de embeddings. | Reduz drasticamente o tamanho (de 110M → 12M). |
| **DeBERTa** | *Disentangled attention* + correção de posição absoluta. | Melhor compreensão sintática e semântica. |
| **BERTimbau** 🇧🇷 | BERT treinado em português (brWac + Wikipedia). | Melhor performance em PT-BR. |

---

## Escolha prática

| Situação | Modelo sugerido |
|-----------|-----------------|
| Poucos recursos de GPU | `distilbert-base-uncased` |
| Tarefa em português | `neuralmind/bert-base-portuguese-cased` |
| Dataset grande e precisão máxima | `roberta-large` ou `deberta-v3-large` |
| Deploy em mobile / produção leve | `tinybert` ou `albert-base-v2` |

## 6.1 — Fine-tuning do BERTimbau (PT-BR) para Classificação de Texto

O **BERTimbau** é o BERT pré-treinado em corpora de Português (Wikipedia + brWac).  
Para tarefas de **classificação** (sentimento, tópicos, intenção, etc.), fazemos **fine-tuning** adicionando uma **camada linear** sobre o vetor de **[CLS]** (internamente, o `pooled_output`).

**Pipeline**
1. Carregar o **tokenizer** e o **modelo** `neuralmind/bert-base-portuguese-cased` (ou *uncased*).
2. Preparar os dados (`text`, `label`).
3. Tokenizar (`[CLS] ... [SEP]`), definir `max_length`.
4. Treinar com **taxa de aprendizado pequena** (ex.: 2e-5), poucas épocas (2–4).
5. Avaliar (accuracy/F1) e testar com frases novas.

**Dicas práticas**
- Use o modelo **cased** para preservar acentuação e caixa em PT-BR.
- Se a base for pequena, considere **congelar** as primeiras camadas do encoder para estabilizar.
- Classes desbalanceadas? Use `class_weights` ou *weighted loss*.
- Métricas: **accuracy** e **F1** (macro/weighted).

**Entradas esperadas pelo código**
- Você pode:
  - (A) informar um **dataset do Hugging Face** com colunas `text` e `label`, ou  
  - (B) apontar para **CSVs** (`train.csv`, `val.csv`) com colunas `text,label`, ou  
  - (C) usar um **mini-dataset didático** embutido (fallback) só para demonstrar.


In [ ]:
# ============================================================
# Fine-tuning BERTimbau (Português) para Classificação de Texto
# ============================================================
import os, math, random, inspect
import numpy as np
import pandas as pd
import torch

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ------------------------------------------------------------
# (Colab) instalar dependências
# ------------------------------------------------------------
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install -U "transformers>=4.39" "datasets>=2.14" "accelerate>=0.28" "evaluate>=0.4" "pandas>=1.5"

# ------------------------------------------------------------
# Imports principais
# ------------------------------------------------------------
import transformers
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer, TrainingArguments
)
print("transformers:", transformers.__version__)

# Helper: TrainingArguments compatível com a sua versão
def build_training_arguments(**kwargs) -> TrainingArguments:
    sig = inspect.signature(TrainingArguments.__init__)
    allowed = set(sig.parameters.keys()); allowed.discard("self")
    filtered = {k: v for k, v in kwargs.items() if k in allowed}
    dropped = [k for k in kwargs if k not in allowed]
    if dropped:
        print("[Aviso] parâmetros ignorados nesta versão:", dropped)
    return TrainingArguments(**filtered)

# ------------------------------------------------------------
# 1) FONTE DOS DADOS (escolha UMA)
# ------------------------------------------------------------
# (A) Dataset Hugging Face (deve ter colunas 'text' e 'label')
HF_DATASET = None
HF_CONFIG  = None  # ex.: "default" ou subtarefa (se houver)

# (B) CSVs locais com colunas: text,label (rótulo pode ser string)
CSV_TRAIN = None  # ex.: "/content/train.csv"
CSV_VAL   = None  # ex.: "/content/val.csv"

# (C) Fallback didático embutido (PT-BR)
FALLBACK_PT = [
    ("o filme é excelente, emocionante e muito bem dirigido.", 1),
    ("péssimo atendimento, não volto mais.", 0),
    ("a comida estava maravilhosa, sabores incríveis.", 1),
    ("produto chegou quebrado e atrasado, experiência horrível.", 0),
    ("serviço rápido e eficiente, gostei bastante.", 1),
    ("interface confusa e cheia de bugs.", 0),
    ("uma experiência fantástica do começo ao fim!", 1),
    ("não recomendo, custo-benefício muito ruim.", 0),
]

# ------------------------------------------------------------
# 2) Carregar dados
# ------------------------------------------------------------
train_texts, train_labels = [], []
val_texts,   val_labels   = [], []

def load_from_hf(name, config=None):
    from datasets import load_dataset
    ds = load_dataset(name, config) if config else load_dataset(name)
    # tenta achar colunas text/label comuns
    # você pode adaptar aqui se seu dataset tiver outros nomes
    def pick_cols(split):
        cand_text = [c for c in ["text", "sentence", "texto", "review", "content"] if c in ds[split].column_names]
        cand_label= [c for c in ["label", "labels", "sentiment", "classe"] if c in ds[split].column_names]
        assert cand_text and cand_label, f"Não encontrei colunas 'text' e 'label' no split {split}. Colunas: {ds[split].column_names}"
        return cand_text[0], cand_label[0]

    tcol_tr, lcol_tr = pick_cols("train")
    tcol_va, lcol_va = pick_cols("validation") if "validation" in ds else pick_cols("test")

    Xtr = ds["train"][tcol_tr];  Ytr = ds["train"][lcol_tr]
    Xva = ds["validation"][tcol_va] if "validation" in ds else ds["test"][tcol_va]
    Yva = ds["validation"][lcol_va] if "validation" in ds else ds["test"][lcol_va]
    return list(Xtr), list(Ytr), list(Xva), list(Yva)

def load_from_csv(path):
    df = pd.read_csv(path)
    assert "text" in df.columns and "label" in df.columns, f"O CSV {path} deve ter colunas: text,label"
    return df["text"].tolist(), df["label"].tolist()

try:
    if HF_DATASET:
        print(f"Carregando dataset HF: {HF_DATASET} ({HF_CONFIG})")
        train_texts, train_labels, val_texts, val_labels = load_from_hf(HF_DATASET, HF_CONFIG)
    elif CSV_TRAIN and CSV_VAL:
        print("Carregando CSVs locais…")
        train_texts, train_labels = load_from_csv(CSV_TRAIN)
        val_texts,   val_labels   = load_from_csv(CSV_VAL)
    else:
        print("Usando fallback didático embutido (PT-BR).")
        pairs = FALLBACK_PT[:]
        random.shuffle(pairs)
        # split 75/25
        n = int(0.75 * len(pairs))
        tr, va = pairs[:n], pairs[n:]
        train_texts = [t for t, y in tr]; train_labels = [y for t, y in tr]
        val_texts   = [t for t, y in va]; val_labels   = [y for t, y in va]
except Exception as e:
    raise RuntimeError(f"Falha ao carregar dados: {e}")

print(f"Tamanho: train={len(train_texts)}  val={len(val_texts)}")

# ------------------------------------------------------------
# 3) Saneamento (garante list[str], remove NaN/None/bytes) + map labels
# ------------------------------------------------------------
def _is_nan(x):
    try: return bool(np.isnan(x))
    except Exception: return False

def to_str(x):
    if x is None: return None
    if isinstance(x, (bytes, bytearray)):
        try: x = x.decode("utf-8", "ignore")
        except Exception: x = str(x)
    if isinstance(x, (np.generic,)): x = x.item()
    if isinstance(x, (float, np.floating)) and _is_nan(x): return None
    s = str(x).strip()
    return s if s else None

def clean_xy(X, y, name="split"):
    Xo, yo = [], []
    bad = 0
    for t, l in zip(X, y):
        s = to_str(t)
        if s is None: bad += 1; continue
        Xo.append(s)
        yo.append(l)
    if bad: print(f"[{name}] {bad} amostras removidas por texto inválido.")
    return Xo, yo

train_texts, train_labels = clean_xy(train_texts, train_labels, "train")
val_texts,   val_labels   = clean_xy(val_texts,   val_labels,   "val")

# Mapear labels (strings → ids)
uniq = sorted({str(l) for l in (list(train_labels) + list(val_labels))})
label2id = {lab:i for i, lab in enumerate(uniq)}
id2label = {i:lab for lab,i in label2id.items()}
train_labels = [label2id[str(l)] for l in train_labels]
val_labels   = [label2id[str(l)] for l in val_labels]
num_labels = len(label2id)
print("Labels:", label2id)

# ------------------------------------------------------------
# 4) Tokenizer (BERTimbau) e tokenização
# ------------------------------------------------------------
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"   # ou "…-uncased"
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

MAX_LEN = 160
def tokenize_batch(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_enc = tokenize_batch(train_texts)
val_enc   = tokenize_batch(val_texts)

class TorchTextDataset(torch.utils.data.Dataset):
    def __init__(self, enc, labels):
        self.enc = enc
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_ds = TorchTextDataset(train_enc, train_labels)
val_ds   = TorchTextDataset(val_enc,   val_labels)

# ------------------------------------------------------------
# 5) Modelo e (opcional) congelamento parcial do encoder
# ------------------------------------------------------------
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
).to(device)

FREEZE_N_LAYERS = 0  # ex.: 6 para congelar 6 camadas iniciais
if FREEZE_N_LAYERS > 0:
    # Congelar embeddings + primeiras N camadas do encoder
    for p in model.bert.embeddings.parameters():
        p.requires_grad = False
    for i in range(FREEZE_N_LAYERS):
        for p in model.bert.encoder.layer[i].parameters():
            p.requires_grad = False
    print(f"Camadas congeladas: embeddings + {FREEZE_N_LAYERS} primeiras camadas.")

# ------------------------------------------------------------
# 6) Métricas (accuracy + F1 se disponível)
# ------------------------------------------------------------
try:
    import evaluate
    acc_metric = evaluate.load("accuracy")
    f1_metric  = evaluate.load("f1")
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        r1 = acc_metric.compute(predictions=preds, references=labels)
        r2 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
        return {"accuracy": r1["accuracy"], "f1": r2["f1"]}
except Exception:
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = (preds == labels).mean()
        return {"accuracy": float(acc)}

# ------------------------------------------------------------
# 7) Treinamento (Trainer)
# ------------------------------------------------------------
EPOCHS = 3 if len(train_texts) >= 1000 else 5
BATCH  = 16 if torch.cuda.is_available() else 8

args = build_training_arguments(
    output_dir="bertimbau-cls-ptbr",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    num_train_epochs=EPOCHS,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    report_to="none",
    seed=SEED
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

print("\n=== Iniciando fine-tuning do BERTimbau ===")
trainer.train()
eval_out = trainer.evaluate()
print("\nResultados de validação:", eval_out)

# ------------------------------------------------------------
# 8) Inferência em frases PT-BR
# ------------------------------------------------------------
def predict(texts):
    model.eval()
    enc = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model(**enc)
        probs = torch.softmax(out.logits, dim=-1).cpu().numpy()
        preds = probs.argmax(axis=-1)
    decoded = [(t, id2label[int(p)], probs[i]) for i,(t,p) in enumerate(zip(texts, preds))]
    return decoded

amostras = [
    "o atendimento foi excelente e rápido.",
    "que decepção, não recomendo a ninguém.",
    "funciona bem, mas poderia ser mais intuitivo."
]
for texto, pred, prob in predict(amostras):
    print(f"- {texto}\n  -> classe: {pred} | probs={np.round(prob, 3)}")

## 6.2 - O que é o `pooled_output` no BERT?


O BERT é um **modelo de codificação de sequência**, ou seja, ele recebe uma lista de tokens e gera um **vetor contextualizado para cada token**.

Por exemplo, uma entrada como:
```text
[CLS] o filme foi ótimo [SEP]
```

gera uma matriz de saída de dimensões:
$begin:math:display$
\\text{last_hidden_state} \\in \\mathbb{R}^{(\\text{seq\\_len} \\times d_{model})}
$end:math:display$
onde cada linha corresponde ao embedding contextual de um token.

---

### Relembrando... O papel do `[CLS]`

O primeiro token especial, `[CLS]` (*classification*), não representa uma palavra real.  
Ele é adicionado **no início da sequência** e serve como um **resumo global da sentença**.

Durante o treinamento, o BERT aprende a "preencher" o vetor do `[CLS]` com informações que sintetizam o significado da sequência inteira.

Assim, o vetor correspondente ao `[CLS]` na saída final é usado como **entrada para tarefas de classificação**, *Next Sentence Prediction*, etc.

---

### Mas afinal... O que é o `pooled_output`?

Depois do *encoder*, o BERT retorna dois valores principais:

1. **`last_hidden_state`** → todos os vetores dos tokens  
   → shape: `(batch_size, seq_len, hidden_size)`  
   → exemplo: `(8, 128, 768)`

2. **`pooled_output`** → vetor único da sequência  
   → shape: `(batch_size, hidden_size)`  
   → exemplo: `(8, 768)`

O `pooled_output` é obtido da seguinte forma:

```python
pooled_output = tanh(W * hidden_state_[CLS] + b)
```

Ou seja:
- Pega-se **somente o vetor do token `[CLS]`** da última camada (`hidden_state_[0]`);
- Passa-se por uma **camada linear** (W, b);
- Aplica-se **tanh** (função de ativação suave);
- O resultado é o **`pooled_output`** — a representação final da sequência.

```text
Saída do encoder (última camada)
↓
[CLS]   O     filme   foi   ótimo   [SEP]
 ↓       ↓       ↓       ↓      ↓
h_cls   h_1     h_2     h_3    h_4
 ↓
Linear + tanh
 ↓
pooled_output (vetor único da sequência)
```

---

### Aplicações

| Tarefa | Usa o quê | Saída |
|--------|------------|-------|
| Classificação de texto | `pooled_output` | 1 vetor por sentença |
| NER / POS tagging | `last_hidden_state` | 1 vetor por token |
| Question answering | `last_hidden_state` | 1 vetor por token (para prever início/fim) |

---

### Dica prática

No `transformers`, quando você roda:

```python
outputs = model(**inputs)
```

você obtém:

```python
outputs.last_hidden_state   # embeddings de todos os tokens
outputs.pooler_output       # o pooled_output (vetor do [CLS])
```

Se você quiser extrair manualmente o vetor `[CLS]` sem o pooling linear:
```python
cls_embedding = outputs.last_hidden_state[:, 0, :]
```

Isso é útil, por exemplo, se quiser testar diferentes *pooling strategies* (média, max, attention-pooling, etc.).

---

## Mostrando `pooled_output` na prática

Vamos fazer o seguinte:
1) Tokenizar algumas frases;
2) Rodar o BERT (BERTimbau) e inspecionar:
   - `last_hidden_state` (um vetor por token),
   - o vetor do `[CLS]` cru (`last_hidden_state[:, 0, :]`),
   - o `pooled_output` (Linear + `tanh` aplicado ao `[CLS]`);
3) Calcular **similaridades cosseno** entre:
   - `[CLS]` cru  ↔ `pooled_output`,
   - `[CLS]` cru  ↔ **mean pooling** (média sobre os tokens válidos),
   - `pooled_output` ↔ **mean pooling**.

> Intuição:
> - O `pooled_output` é uma **transformação não-linear** do `[CLS]` (Linear + `tanh`);
> - Às vezes, **mean pooling** de todos os tokens (ignorando `PAD`) funciona melhor em algumas tarefas — é bom comparar.

In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizerFast, BertModel

# Se quiser trocar por "bert-base-uncased", basta alterar o nome:
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME).to(device).eval()

sentences = [
    "O filme foi excelente e muito emocionante!",
    "O atendimento foi péssimo e me deixou insatisfeito.",
    "Funciona bem, mas poderia ser mais rápido.",
]

# ---------------------------
# Tokenização + forward
# ---------------------------
enc = tokenizer(
    sentences,
    padding=True,
    truncation=True,
    max_length=64,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    outputs = model(**enc, return_dict=True)
    last_hidden = outputs.last_hidden_state               # (B, T, H)
    pooled_output = outputs.pooler_output                 # (B, H) = tanh(W * h_cls + b)

# ---------------------------
# Extrair o vetor [CLS] cru (posição 0) e mean pooling
# ---------------------------
cls_raw = last_hidden[:, 0, :]                            # (B, H)

# mean pooling com máscara (ignora PAD)
mask = enc["attention_mask"].unsqueeze(-1).float()        # (B, T, 1)
sum_tokens = (last_hidden * mask).sum(dim=1)              # (B, H)
len_tokens = mask.sum(dim=1).clamp(min=1e-6)              # (B, 1) evita div/0
mean_pool = sum_tokens / len_tokens                       # (B, H)

# ---------------------------
# Similaridades cosseno para comparar representações
# ---------------------------
def cos(a, b):
    return F.cosine_similarity(a, b, dim=-1).detach().cpu()

sim_cls_pooled = cos(cls_raw, pooled_output)              # (B,)
sim_cls_mean   = cos(cls_raw, mean_pool)                  # (B,)
sim_pool_mean  = cos(pooled_output, mean_pool)            # (B,)

# (Opcional) Reaplicar a pooler manualmente (quando disponível) para mostrar equivalência
recomputed_ok = False
try:
    with torch.no_grad():
        # algumas versões expõem a pooler como model.pooler ou model.bert.pooler
        pool = getattr(model, "pooler", None) or getattr(model, "bert", None).pooler
        pooled_re = pool(last_hidden)                     # (B, H)
        diff = (pooled_re - pooled_output).abs().max().item()
        recomputed_ok = diff < 1e-6
except Exception:
    pass

# ---------------------------
# Exibir resultados
# ---------------------------
print(f"Device: {device}")
print("Shapes:")
print("  last_hidden_state:", tuple(last_hidden.shape))
print("  [CLS] raw        :", tuple(cls_raw.shape))
print("  pooled_output    :", tuple(pooled_output.shape))
print("  mean_pool        :", tuple(mean_pool.shape))

print("\nSimilaridades cosseno (por amostra):")
for i, s in enumerate(sentences):
    print(f"\nFrase {i+1}: {s}")
    print(f"  cos([CLS], pooled) = {sim_cls_pooled[i]:.4f}")
    print(f"  cos([CLS], mean)   = {sim_cls_mean[i]:.4f}")
    print(f"  cos(pooled, mean)  = {sim_pool_mean[i]:.4f}")

if recomputed_ok:
    print("\n[OK] pooler interno reaplicado == pooled_output (diferença < 1e-6).")
else:
    print("\n[INFO] Não foi possível (ou não faz sentido nesta versão) reaplicar a pooler internamente.")

## 6.3 - DistilBERT e a técnica de Distilação de Conhecimento

Modelos BERT são poderosos, mas muito pesados:  
o **BERT-base** tem cerca de **110 milhões de parâmetros**, exigindo grande custo de memória e tempo de inferência.

Para aplicações em tempo real (chatbots, busca, mobile), isso é um gargalo.  
O **DistilBERT** foi criado como uma **versão compacta do BERT**, mantendo a maior parte do desempenho com metade do tamanho.

---

### O que é Distilação de Conhecimento

A ideia vem de *Knowledge Distillation* (Hinton et al., 2015):  
transferir o “conhecimento” de um modelo grande (*teacher*) para um modelo menor (*student*).

O processo segue 3 etapas:

1. **Treinar o professor** (ex.: BERT-base) normalmente.
2. **Treinar o aluno** (DistilBERT) usando:
   - As **saídas reais** do professor (probabilidades sobre as classes ou tokens);
   - E as **saídas intermediárias** (como embeddings e atenções) para que o aluno aprenda a imitá-las.

A perda total combina três termos:

$$
\mathcal{L} = \alpha_{\text{soft}} \cdot \text{CE}(y_{\text{teacher}}, y_{\text{student}}) + \alpha_{\\text{hard}} \cdot \text{CE}(y_{\text{true}}, y_{\text{student}}) + \alpha_{\text{hidden}} \cdot \|h_{\text{teacher}} - h_{\text{student}}\|^2
$$

onde:
- **soft loss** → distilação entre distribuições suavizadas (`softmax` com temperatura $begin:math:text$T>1$end:math:text$);  
- **hard loss** → erro normal de predição;  
- **hidden loss** → aproximação entre embeddings internos.

---

### Entendendo a função de perda do DistilBERT

A distilação de conhecimento combina **três tipos de aprendizado** — supervisionado, por imitação e estrutural — em uma única função de custo.

A função total é:

$$
\mathcal{L} =
\alpha_{\text{soft}} \cdot \text{CE}(y_{\text{teacher}}, y_{\text{student}})
+ \alpha_{\text{hard}} \cdot \text{CE}(y_{\text{true}}, y_{\text{student}})
+ \alpha_{\text{hidden}} \cdot \|h_{\text{teacher}} - h_{\text{student}}\|^2
$$

onde:
- **CE** = *Cross-Entropy Loss*  
- $y_{\text{teacher}}$: saída (probabilidades) do modelo professor  
- $y_{\text{student}}$: saída (probabilidades) do modelo aluno  
- $y_{\text{true}}$: rótulo real  
- $h_{\text{teacher}}, h_{\text{student}}$: vetores das camadas internas (*hidden states*)  
- $\alpha_{\text{soft}}, \alpha_{\text{hard}}, \alpha_{\text{hidden}}$: pesos de cada termo  

---

#### Primeiro termo — *Soft Loss* (Imitação do Professor)

$$
\alpha_{\text{soft}} \cdot \text{CE}(y_{\text{teacher}}, y_{\text{student}})
$$

O aluno aprende a **imitar a distribuição de probabilidades** do professor, e não apenas o rótulo final.

O *teacher* gera uma distribuição de probabilidades sobre o vocabulário (via *softmax*).  
Essas probabilidades são “suavizadas” com uma **temperatura $T > 1$**:

$$
p_i = \frac{e^{z_i / T}}{\sum_j e^{z_j / T}}
$$

Valores maiores de $T$ tornam a distribuição menos “dura”, expondo mais *informação relacional* — por exemplo, o professor mostra que "ótimo" e "excelente" são parecidos, mas "péssimo" é muito diferente.

Assim, o aluno aprende:
> “como o professor pensa”, não apenas “qual classe ele escolheu”.

---

#### Segundo termo — *Hard Loss* (Supervisão tradicional)

$$
\alpha_{\text{hard}} \cdot \text{CE}(y_{\text{true}}, y_{\text{student}})
$$

É a **perda normal de classificação**, usando os rótulos verdadeiros do dataset.  
Esse termo garante que o aluno continue aprendendo a tarefa original enquanto imita o professor.

---

#### Terceiro termo — *Hidden-State Alignment Loss*

$$
\alpha_{\text{hidden}} \cdot \|h_{\text{teacher}} - h_{\text{student}}\|^2
$$

Além de copiar as saídas finais, o DistilBERT também aprende a **replicar as representações internas** do BERT.

Durante o pré-treinamento:
- as camadas do aluno são alinhadas com camadas equivalentes do professor;
- o aluno tenta minimizar a **distância L2** entre embeddings correspondentes.

Esse termo faz o aluno “pensar” de maneira parecida, camada a camada.

---

#### Combinando os termos

| Termo | Tipo de aprendizado | Papel no treino |
|--------|---------------------|-----------------|
| $\alpha_{\text{soft}} \cdot CE(y_t, y_s)$ | Imitação (*distilação*) | Fazer o aluno reproduzir o comportamento do professor |
| $\alpha_{\text{hard}} \cdot CE(y_{true}, y_s)$ | Supervisionado | Garantir que o aluno continue resolvendo a tarefa original |
| $\alpha_{\text{hidden}} \cdot \|h_t - h_s\|^2$ | Estrutural | Fazer o aluno representar internamente o conhecimento do professor |


```text
        ┌─────────────┐
        │  Teacher    │
        │ (BERT-base) │
        └────┬────────┘
             │
             │  y_teacher, h_teacher
             ▼
        ┌─────────────┐
        │  Student    │
        │ (DistilBERT)│
        └────┬────────┘
             │
     ┌───────┴──────────────────────────────┐
     │ 3 perdas combinadas:                 │
     │   1) Soft → imitar distribuições     │
     │   2) Hard → prever rótulo correto    │
     │   3) Hidden → copiar representações  │
     └──────────────────────────────────────┘
```

---

#### Valores típicos

No treinamento original do DistilBERT (Sanh et al., 2019):

- $T = 2.0$  
- $\alpha_{\text{soft}} = 0.5$  
- $\alpha_{\text{hard}} = 0.5$  
- $\alpha_{\text{hidden}} = 1.0$

Esses valores equilibram **imitação** e **fidelidade à tarefa**.

#### Em resumo...

> O DistilBERT aprende não apenas com **respostas finais**, mas com **o raciocínio interno do professor**.  
> Ele tenta ser um aluno mais rápido, mas com o mesmo “jeito de pensar”.

---

### Como o DistilBERT é treinado

O DistilBERT (Sanh et al., 2019) segue esta configuração:

| Item | BERT-base | DistilBERT |
|------|------------|------------|
| Camadas (encoder) | 12 | 6 |
| Cabeças de atenção | 12 | 12 |
| Hidden size | 768 | 768 |
| Parâmetros | 110M | 66M |
| Velocidade | — | 60% mais rápido |
| Tamanho | — | 40% menor |

O **student** é inicializado com **camadas alternadas do teacher**:  
as camadas 2, 4, 6, 8, 10, 12 do BERT são copiadas.

Durante o pré-treinamento:
- o *teacher* (BERT-base) fica congelado;
- o *student* aprende:
  - **Masked Language Modeling (MLM)**,  
  - **Distilação de logits** (soft targets do teacher),  
  - e **similaridade entre estados ocultos**.

O DistilBERT **não possui o token [CLS] pooler nem a cabeça NSP (Next Sentence Prediction)**.  
Ou seja, ele é otimizado apenas para o objetivo de **MLM + distilação**.

---

#### Desempenho

Mesmo com metade das camadas, o DistilBERT mantém cerca de **97% da acurácia do BERT-base** em benchmarks como GLUE, com:
- 40% menos parâmetros,
- 60% menos custo computacional,
- 2× mais rápido na inferência.

```text
        ┌──────────────────────────────────────────┐
        │               BERT-base                  │
        │ 12 camadas, 110M params (teacher)        │
        └───────┬──────────────┬───────────────┬───┘
                │              │               │
                ▼              ▼               ▼
        ┌──────────────────────────────────────────┐
        │              DistilBERT                  │
        │ 6 camadas, 66M params (student)          │
        └──────────────────────────────────────────┘
                 ↑ aprende com ↓
     (soft logits + hidden states + labels reais)
```

- **DistilBERT = BERT menor + mesmo vocabulário + sem NSP**
- **Treinado com distilação de conhecimento**
- **Resultado:** mais leve, mais rápido, quase mesma performance

In [ ]:
# ============================================================
# Fine-tuning DistilBERT para Classificação de Texto
# ============================================================
import os, math, random, inspect
import numpy as np
import pandas as pd
import torch

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

# ------------------------------------------------------------
# (Colab) instalar dependências
# ------------------------------------------------------------
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install -U "transformers>=4.39" "datasets>=2.14" "accelerate>=0.28" "evaluate>=0.4" "pandas>=1.5"

# ------------------------------------------------------------
# Imports principais
# ------------------------------------------------------------
import transformers
from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer, TrainingArguments
)
print("transformers:", transformers.__version__)

# Helper: TrainingArguments compatível com sua versão
def build_training_arguments(**kwargs) -> TrainingArguments:
    sig = inspect.signature(TrainingArguments.__init__)
    allowed = set(sig.parameters.keys()); allowed.discard("self")
    filtered = {k: v for k, v in kwargs.items() if k in allowed}
    dropped = [k for k in kwargs if k not in allowed]
    if dropped:
        print("[Aviso] parâmetros ignorados nesta versão:", dropped)
    return TrainingArguments(**filtered)

# ------------------------------------------------------------
# 1) Escolha da FONTE DE DADOS (selecione UMA)
# ------------------------------------------------------------
# (A) Dataset Hugging Face (deve ter colunas text/label ou similares)
HF_DATASET = "yelp_polarity"  # ou "imdb", "ag_news", etc. ou None
HF_CONFIG  = None

# (B) CSVs locais com colunas: text,label
CSV_TRAIN = None  # ex: "/content/train.csv"
CSV_VAL   = None  # ex: "/content/val.csv"

# (C) Fallback didático (PT/EN misto, só p/ demo)
FALLBACK = [
    ("o filme é excelente e muito bem dirigido", 1),
    ("péssimo atendimento, não volto mais", 0),
    ("the product is amazing and works great", 1),
    ("awful experience, totally disappointed", 0),
    ("serviço rápido e eficiente, gostei", 1),
    ("interface confusa e cheia de bugs", 0),
]

# ------------------------------------------------------------
# 2) Carregar dados
# ------------------------------------------------------------
train_texts, train_labels = [], []
val_texts,   val_labels   = [], []

def load_from_hf(name, config=None):
    from datasets import load_dataset
    ds = load_dataset(name, config) if config else load_dataset(name)
    # tenta detectar colunas
    def pick_cols(split):
        cols = ds[split].column_names
        cand_text = [c for c in ["text", "sentence", "review", "content"] if c in cols] or [cols[0]]
        cand_label= [c for c in ["label", "labels", "sentiment", "stars"] if c in cols] or [cols[1]]
        return cand_text[0], cand_label[0]
    t_tr, l_tr = pick_cols("train")
    t_va, l_va = pick_cols("test") if "validation" not in ds else pick_cols("validation")
    Xtr = ds["train"][t_tr];  Ytr = ds["train"][l_tr]
    Xva = ds["test"][t_va] if "validation" not in ds else ds["validation"][t_va]
    Yva = ds["test"][l_va] if "validation" not in ds else ds["validation"][l_va]
    return list(Xtr), list(Ytr), list(Xva), list(Yva)

def load_from_csv(path):
    df = pd.read_csv(path)
    assert "text" in df.columns and "label" in df.columns, f"O CSV {path} deve ter colunas: text,label"
    return df["text"].tolist(), df["label"].tolist()

try:
    if HF_DATASET:
        print(f"Carregando dataset HF: {HF_DATASET} ({HF_CONFIG})")
        train_texts, train_labels, val_texts, val_labels = load_from_hf(HF_DATASET, HF_CONFIG)
    elif CSV_TRAIN and CSV_VAL:
        print("Carregando CSVs locais…")
        train_texts, train_labels = load_from_csv(CSV_TRAIN)
        val_texts,   val_labels   = load_from_csv(CSV_VAL)
    else:
        print("Usando fallback didático embutido.")
        pairs = FALLBACK[:]
        random.shuffle(pairs)
        n = int(0.7 * len(pairs))
        tr, va = pairs[:n], pairs[n:]
        train_texts = [t for t,_ in tr]; train_labels = [y for _,y in tr]
        val_texts   = [t for t,_ in va]; val_labels   = [y for _,y in va]
except Exception as e:
    raise RuntimeError(f"Falha ao carregar dados: {e}")

print(f"Tamanho: train={len(train_texts)}  val={len(val_texts)}")

# ------------------------------------------------------------
# 3) Saneamento (garante list[str], remove NaN/None/bytes) + map de labels
# ------------------------------------------------------------
def _is_nan(x):
    try: return bool(np.isnan(x))
    except Exception: return False

def to_str(x):
    if x is None: return None
    if isinstance(x, (bytes, bytearray)):
        try: x = x.decode("utf-8", "ignore")
        except Exception: x = str(x)
    if isinstance(x, (np.generic,)): x = x.item()
    if isinstance(x, (float, np.floating)) and _is_nan(x): return None
    s = str(x).strip()
    return s if s else None

def clean_xy(X, y, name="split"):
    Xo, yo = [], []
    bad = 0
    for t, l in zip(X, y):
        s = to_str(t)
        if s is None: bad += 1; continue
        Xo.append(s); yo.append(l)
    if bad: print(f"[{name}] {bad} amostras removidas por texto inválido.")
    return Xo, yo

train_texts, train_labels = clean_xy(train_texts, train_labels, "train")
val_texts,   val_labels   = clean_xy(val_texts,   val_labels,   "val")

# Mapeia labels (strings → ids)
uniq = sorted({str(l) for l in (list(train_labels)+list(val_labels))})
label2id = {lab:i for i,lab in enumerate(uniq)}
id2label = {i:lab for lab,i in label2id.items()}
train_labels = [label2id[str(l)] for l in train_labels]
val_labels   = [label2id[str(l)] for l in val_labels]
num_labels = len(label2id)
print("Labels:", label2id)

# ------------------------------------------------------------
# 4) Tokenizer (DistilBERT) e tokenização
# ------------------------------------------------------------
MODEL_NAME = "distilbert-base-uncased"    # troque aqui se quiser outro DistilBERT
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

MAX_LEN = 160
def tokenize_batch(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )

train_enc = tokenize_batch(train_texts)
val_enc   = tokenize_batch(val_texts)

class TorchTextDataset(torch.utils.data.Dataset):
    def __init__(self, enc, labels):
        self.enc = enc; self.labels = labels
    def __len__(self): return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.enc.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_ds = TorchTextDataset(train_enc, train_labels)
val_ds   = TorchTextDataset(val_enc,   val_labels)

# ------------------------------------------------------------
# 5) Modelo
# ------------------------------------------------------------
model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
).to(device)

# ------------------------------------------------------------
# 6) Métricas (accuracy + F1 se disponível)
# ------------------------------------------------------------
try:
    import evaluate
    acc_metric = evaluate.load("accuracy")
    f1_metric  = evaluate.load("f1")
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        r1 = acc_metric.compute(predictions=preds, references=labels)
        r2 = f1_metric.compute(predictions=preds, references=labels, average="weighted")
        return {"accuracy": r1["accuracy"], "f1": r2["f1"]}
except Exception:
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = (preds == labels).mean()
        return {"accuracy": float(acc)}

# ------------------------------------------------------------
# 7) Treinamento (Trainer)
# ------------------------------------------------------------
EPOCHS = 2 if len(train_texts) > 1000 else 4
BATCH  = 16 if torch.cuda.is_available() else 8

args = build_training_arguments(
    output_dir="distilbert-cls",
    evaluation_strategy="epoch",
    save_strategy="no",
    learning_rate=3e-5,          # DistilBERT costuma aceitar 3e-5 bem
    weight_decay=0.01,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    num_train_epochs=EPOCHS,
    fp16=torch.cuda.is_available(),
    logging_steps=50,
    report_to="none",
    seed=SEED
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

print("\n=== Iniciando fine-tuning do DistilBERT ===")
trainer.train()
eval_out = trainer.evaluate()
print("\nResultados de validação:", eval_out)

# ------------------------------------------------------------
# 8) Inferência em frases
# ------------------------------------------------------------
def predict(texts):
    model.eval()
    enc = tokenizer(texts, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt").to(device)
    with torch.no_grad():
        out = model(**enc)
        probs = torch.softmax(out.logits, dim=-1).cpu().numpy()
        preds = probs.argmax(axis=-1)
    decoded = [(t, id2label[int(p)], probs[i]) for i,(t,p) in enumerate(zip(texts, preds))]
    return decoded

samples = [
    "The product quality is amazing and the delivery was fast.",
    "Horrible support. I will never buy again.",
    "serviço excelente e muito rápido, recomendo",
    "não gostei, veio com defeito."
]
for texto, pred, prob in predict(samples):
    print(f"- {texto}\n  -> classe: {pred} | probs={np.round(prob, 3)}")